In [1]:
import numpy as np
from mp_api.client import MPRester
from pymatgen.core.operations import SymmOp
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.electronic_structure.plotter import BSPlotter
from pymatgen.phonon.plotter import PhononBSPlotter
from jupyter_jsmol.pymatgen import quick_view
from lmapr1492 import plot_brillouin_zone, get_plot_bs, get_plot_dos, get_plot_bs_and_dos, get_branch_wavevectors
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [2]:
mp_key = "VGJDyi0TH2uezP850Utn6gEwHnI5i365"
mp_id = "mp-961713" #ZrSnPt
with MPRester(mp_key) as m:
    prim_struc = m.get_structure_by_material_id(mp_id)
    el_bs = m.get_bandstructure_by_material_id(mp_id)
    el_dos = m.get_dos_by_material_id(mp_id)
    ph_bs = m.get_phonon_bandstructure_by_material_id(mp_id)
    ph_dos = m.get_phonon_dos_by_material_id(mp_id)
conv_struc = SpacegroupAnalyzer(prim_struc).get_conventional_standard_structure()
symmops = SpacegroupAnalyzer(conv_struc).get_space_group_operations()

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving ElectronicStructureDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving ElectronicStructureDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving PhononBSDOSDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving PhononBSDOSDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

# Première opération de symétrie (Rotation d'ordre 2)

In [18]:
i_atom = 1 #Zr0
i_symmop = 3

view = quick_view(prim_struc, "packed", conventional = True)
display(view)
view.script('select zirconium; color lightsalmon')
view.script('select tin; color gold')
view.script('select platinum; color palegreen')
view.script('draw SYMOP ' + str(i_symmop) + ' {atomno = ' + str(i_atom) + '}')


JsmolView(layout=Layout(align_self='stretch', height='400px'))

In [4]:
symmop = symmops[i_symmop - 1]
print(symmop)

Rot:
[[-1.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0.  1.]]
tau
[0. 0. 0.]


In [5]:
pos_init = conv_struc.sites[i_atom -1].frac_coords
print(pos_init)

[0.5 0.  0. ]


In [6]:
pos_final = symmop.operate(pos_init)
print(pos_final)

[-0.5  0.   0. ]


In [7]:
for k_atom, site in enumerate(conv_struc.sites):
    if np.linalg.norm((site.frac_coords - pos_final)%1) < 1e-6:
        j_atom = k_atom + 1
print(j_atom, conv_struc.sites[j_atom -1].frac_coords)

1 [0.5 0.  0. ]


### Description :
On applique une rotation d'ordre 2 sans translation sur l'atome Zr0 qui passe de la position $\begin{bmatrix} 0.5 & 0 & 0 \end{bmatrix}$ à la position $\begin{bmatrix} -0.5 & 0 & 0 \end{bmatrix}$.

L'équation de cette symétrie est:
$$
\begin{bmatrix}
-1 & 0 & 0 \\
0 & -1 & 0 \\
0 & 0 & 1
\end{bmatrix}
\begin{bmatrix}
x \\
y \\
z
\end{bmatrix}
=
\begin{bmatrix}
-x \\
-y \\
z
\end{bmatrix}\\
$$
Cette opération conserve la chiralité et tourne l'atome de 180° autour d'un axe (celui entouré de la flèche rouge sur l'image)

# Deuxième opération de symétrie (Réflexion)

In [19]:
i_atom = 6 #Sn5
i_symmop = 8

view = quick_view(prim_struc, "packed", conventional = True)
display(view)
view.script('select zirconium; color lightsalmon')
view.script('select tin; color gold')
view.script('select platinum; color palegreen')
view.script('draw SYMOP ' + str(i_symmop) + ' {atomno = ' + str(i_atom) + '}')

JsmolView(layout=Layout(align_self='stretch', height='400px'))

In [9]:
symmop = symmops[i_symmop - 1]
print(symmop)

Rot:
[[ 0. -1.  0.]
 [-1.  0.  0.]
 [ 0.  0.  1.]]
tau
[0. 0. 0.]


In [10]:
pos_init = conv_struc.sites[i_atom -1].frac_coords
print(pos_init)

[0.  0.5 0.5]


In [11]:
pos_final = symmop.operate(pos_init)
print(pos_final)

[-0.5  0.   0.5]


In [12]:
for k_atom, site in enumerate(conv_struc.sites):
    if np.linalg.norm((site.frac_coords - pos_final)%1) < 1e-6:
        j_atom = k_atom + 1
print(j_atom, conv_struc.sites[j_atom -1].frac_coords)

7 [0.5 0.  0.5]


### Description :
On applique une réflexion sans translation sur l'atome Sn5 qui passe de la position $\begin{bmatrix} 0 & 0.5 & 0.5 \end{bmatrix}$ à la position $\begin{bmatrix} -0.5 & 0 & 0.5 \end{bmatrix}$.

L'équation de cette symétrie est:
$$
\begin{bmatrix}
0 & -1 & 0 \\
-1 & 0 & 0 \\
0 & 0 & 1
\end{bmatrix}
\begin{bmatrix}
x \\
y \\
z
\end{bmatrix}
=
\begin{bmatrix}
-y \\
-x \\
z
\end{bmatrix}\\
$$
Cette opération inverse la chiralité et se fait par rapport à un plan miroir (en vert sur l'image)

# Troisième opération de symétrie (Rotation d'ordre 3)

In [20]:
i_atom = 10 #Pt9
i_symmop = 17

view = quick_view(prim_struc, "packed", conventional = True)
display(view)
view.background_color = (1, 1, 1)
view.script('select zirconium; color lightsalmon')
view.script('select tin; color gold')
view.script('select platinum; color palegreen')
view.script('draw SYMOP ' + str(i_symmop) + ' {atomno = ' + str(i_atom) + '}')

JsmolView(layout=Layout(align_self='stretch', height='400px'))

In [14]:
symmop = symmops[i_symmop - 1]
print(symmop)

Rot:
[[0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]]
tau
[0. 0. 0.]


In [15]:
pos_init = conv_struc.sites[i_atom -1].frac_coords
print(pos_init)

[0.75 0.75 0.75]


In [16]:
pos_final = symmop.operate(pos_init)
print(pos_final)

[0.75 0.75 0.75]


In [17]:
for k_atom, site in enumerate(conv_struc.sites):
    if np.linalg.norm((site.frac_coords - pos_final)%1) < 1e-6:
        j_atom = k_atom + 1
print(j_atom, conv_struc.sites[j_atom -1].frac_coords)

10 [0.75 0.75 0.75]


### Description :
On applique une rotation d'ordre 3 sans translation sur l'atome Pt9 qui passe de la position $\begin{bmatrix} 0.75 & 0.75 & 0.75 \end{bmatrix}$ à la position $\begin{bmatrix} 0.75 & -0.75 & -0.75 \end{bmatrix}$.

L'équation de cette symétrie est:
$$
\begin{bmatrix}
0 & 1 & 0 \\
0 & 0 & 1 \\
1 & 0 & 0
\end{bmatrix}
\begin{bmatrix}
x \\
y \\
z
\end{bmatrix}
=
\begin{bmatrix}
y \\
z \\
x
\end{bmatrix}\\
$$
Cette opération conserve la chiralité et tourne l'atome de 120° autour d'un axe (celui entouré de la flèche rouge sur l'image)